https://github.com/lsjsj92/recommender_system_with_Python/blob/master/004.%20recommender%20system%20basic%20with%20Python%20-%203%20Matrix%20Factorization.ipynb

In [1]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 문제 정보만 가져온다
problem_data = pd.read_csv("preprocessing_data/recommend_problem_infos.csv")

problem_data = problem_data.iloc[:,:11]
problem_data['problem_id'] = range(len(problem_data))

problem_data.info()
problem_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9267 entries, 0 to 9266
Data columns (total 12 columns):
number            9267 non-null int64
name              9267 non-null object
information       9267 non-null object
correct_user      9267 non-null int64
submission_cnt    9267 non-null int64
correct_rate      9267 non-null float64
level             9267 non-null int64
avg_try           9267 non-null float64
time_limit        9267 non-null object
memory_limit      9267 non-null object
algos             9267 non-null object
problem_id        9267 non-null int32
dtypes: float64(2), int32(1), int64(4), object(5)
memory usage: 832.7+ KB


,number,name,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit,algos,problem_id
0,1000,A+B,"다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB,"수학,구현,사칙연산",0
1,1001,A-B,"디버그,분류",97109,159168,72.049,1,1.39,2 초,128 MB,"수학,구현,사칙연산",1
2,1002,터렛,분류,16141,105843,20.562,7,4.86,2 초,128 MB,"수학,기하학",2
3,1003,피보나치 함수,분류,23293,113982,30.087,8,3.32,0.25 초 (추가 시간 없음),128 MB,다이나믹 프로그래밍,3
4,1004,어린 왕자,분류,6503,19933,41.555,8,2.41,2 초,128 MB,기하학,4


In [3]:
recommend_problem_df = pd.read_csv("final_data/recommend_problems.csv")

recommend_problem_df.info()
recommend_problem_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6148089 entries, 0 to 6148088
Data columns (total 7 columns):
id            int64
user_id       int64
user          object
number        int64
TF            object
problem_id    int64
score         int64
dtypes: int64(5), object(2)
memory usage: 328.3+ MB


,id,user_id,user,number,TF,problem_id,score
0,0,0,koosaga,1000,o,0,1
1,1,0,koosaga,1001,o,1,1
2,2,0,koosaga,1002,o,2,1
3,3,0,koosaga,1003,o,3,1
4,4,0,koosaga,1004,o,4,1


In [4]:
user2idx = {}
for user_id, user_name in zip(recommend_problem_df['user_id'], recommend_problem_df['user']) :
    user2idx[user_name] = user_id
len(user2idx)

34900

In [5]:
problem2idx = {}
for index, number in enumerate(recommend_problem_df['number'].unique()) :
    problem2idx[index] = number
len(problem2idx)

9267

In [6]:
len(recommend_problem_df['problem_id'].unique()), len(recommend_problem_df['number'].unique())

(9267, 9267)

In [7]:
score_data = recommend_problem_df[['user_id', 'problem_id', 'score']]

score_data.info()
score_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6148089 entries, 0 to 6148088
Data columns (total 3 columns):
user_id       int64
problem_id    int64
score         int64
dtypes: int64(3)
memory usage: 140.7 MB


,user_id,problem_id,score
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1


In [8]:
user_problem_data = pd.merge(score_data, problem_data, on = 'problem_id')
user_problem_data.head()

,user_id,problem_id,score,number,name,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit,algos
0,0,0,1,1000,A+B,"다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB,"수학,구현,사칙연산"
1,1,0,1,1000,A+B,"다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB,"수학,구현,사칙연산"
2,2,0,1,1000,A+B,"다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB,"수학,구현,사칙연산"
3,3,0,1,1000,A+B,"다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB,"수학,구현,사칙연산"
4,4,0,1,1000,A+B,"다국어,디버그,분류",118169,381174,43.753,1,2.29,2 초,128 MB,"수학,구현,사칙연산"


In [9]:
user_problem_data.shape

(6148089, 14)

In [10]:
user_problem_score = user_problem_data.pivot_table('score', index = 'user_id', columns = 'number').fillna(0)
user_problem_score.shape

(34900, 9267)

In [11]:
user_problem_score.head()

number,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,21215,21221,21222,21226,21227,21232,21233,21236,21262,21269
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
problem_user_score = user_problem_score.values.T
problem_user_score.shape

(9267, 34900)

In [13]:
SVD = TruncatedSVD(n_components=12)
matrix = SVD.fit_transform(problem_user_score)
matrix.shape

(9267, 12)

In [14]:
corr = np.corrcoef(matrix)
corr.shape

(9267, 9267)

In [15]:
problem_number = user_problem_score.columns
problem_number_list = list(problem_number)
nNm = problem_number_list.index(15651)

In [16]:
corr_nNm  = corr[nNm]
list(problem_number[(corr_nNm >= 0.9)])

[2580, 9663, 12865, 15649, 15650, 15651, 15652, 17298]

In [17]:
score_data.head()

,user_id,problem_id,score
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1


In [18]:
df_user_problem_scores = score_data.pivot(index = "user_id", columns = "problem_id", values = 'score').fillna(0)
df_user_problem_scores.head()

problem_id,0,1,2,3,4,5,6,7,8,9,...,9257,9258,9259,9260,9261,9262,9263,9264,9265,9266
user_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
matrix = df_user_problem_scores.as_matrix()

user_scores_mean = np.mean(matrix, axis = 1)

matrix_user_mean = matrix - user_scores_mean.reshape(-1, 1)

In [20]:
matrix

array([[1., 1., 1., ..., 0., 0., 1.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [21]:
pd.DataFrame(matrix_user_mean, columns = df_user_problem_scores.columns).head()

problem_id,0,1,2,3,4,5,6,7,8,9,...,9257,9258,9259,9260,9261,9262,9263,9264,9265,9266
0,0.304414,0.304414,0.304414,0.304414,0.304414,0.304414,0.304414,0.304414,0.304414,0.304414,...,-0.695586,-0.695586,-0.695586,0.304414,-0.695586,-0.695586,-0.695586,-0.695586,-0.695586,0.304414
1,0.378116,0.378116,0.378116,0.378116,0.378116,0.378116,0.378116,0.378116,0.378116,0.378116,...,-0.621884,-0.621884,-0.621884,-0.621884,-0.621884,-0.621884,-0.621884,-0.621884,-0.621884,-0.621884
2,0.475774,0.475774,0.475774,0.475774,0.475774,0.475774,0.475774,0.475774,0.475774,0.475774,...,0.475774,-0.524226,-0.524226,-0.524226,-0.524226,-0.524226,-0.524226,-0.524226,-0.524226,-0.524226
3,0.500378,0.500378,0.500378,0.500378,0.500378,0.500378,0.500378,0.500378,0.500378,0.500378,...,-0.499622,-0.499622,-0.499622,-0.499622,-0.499622,-0.499622,-0.499622,-0.499622,-0.499622,-0.499622
4,0.501781,0.501781,0.501781,0.501781,0.501781,0.501781,0.501781,0.501781,0.501781,0.501781,...,-0.498219,-0.498219,-0.498219,-0.498219,-0.498219,-0.498219,-0.498219,-0.498219,-0.498219,-0.498219


In [22]:
U, sigma, Vt = svds(matrix_user_mean, k = 12)

In [23]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(34900, 12)
(12,)
(12, 9267)


In [24]:
sigma = np.diag(sigma)

In [25]:
sigma.shape

(12, 12)

In [26]:
sigma[0], sigma[1]

(array([178.85121295,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ]),
 array([  0.        , 198.50825048,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ]))

In [27]:
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_scores_mean.reshape(-1, 1)

In [28]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_user_problem_scores.columns)
df_svd_preds.head()

problem_id,0,1,2,3,4,5,6,7,8,9,...,9257,9258,9259,9260,9261,9262,9263,9264,9265,9266
0,1.092643,1.161317,0.940160,0.995191,1.091377,1.296092,0.955534,0.959898,1.225682,1.054703,...,0.593172,0.594564,0.592707,0.592895,0.591426,0.591660,0.592336,0.593313,0.594595,0.593421
1,1.199842,1.209776,0.630115,0.485913,0.951153,1.372896,0.978107,1.021602,1.216834,0.589947,...,0.443747,0.446063,0.442593,0.442180,0.439982,0.440925,0.441569,0.443817,0.445833,0.443297
2,1.164612,1.132237,0.682601,0.727076,0.859683,1.087092,0.776338,0.829634,1.103832,0.679217,...,0.357684,0.360417,0.356749,0.356056,0.354249,0.355035,0.355652,0.357928,0.359590,0.357017
3,1.027797,1.066543,0.969558,0.663756,1.052454,1.456662,0.840841,0.875087,1.093395,0.980097,...,0.342221,0.343483,0.341485,0.341329,0.339838,0.340599,0.340779,0.342156,0.343736,0.342085
4,1.153425,1.177639,0.611948,0.251381,0.955108,1.506327,0.943701,1.001102,1.193697,0.565558,...,0.279569,0.282073,0.278250,0.277779,0.275266,0.276456,0.277055,0.279586,0.282008,0.279084


In [29]:
df_svd_preds.shape

(34900, 9267)

In [30]:
def recommend_problems(df_svd_preds, user_id, ori_problem_df, ori_scores_df, num_recommendations = 5):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.iloc[user_id].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_scores_df[ori_scores_df.user_id == user_id]
    
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합친다. 
    user_history = user_data.merge(ori_problem_df, on = 'problem_id').sort_values(['score'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = ori_problem_df[~ori_problem_df['problem_id'].isin(user_history['problem_id'])]
    
    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'problem_id')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [31]:
already_rated, predictions = recommend_problems(df_svd_preds, 5258, problem_data, score_data, 10)

In [32]:
already_rated.head(10)

,user_id,problem_id,score,number,name,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit,algos
255,5258,6357,2,15998,카카오머니,"스페셜 저지,출처,분류",184,3131,10.127,12,9.88,5 초 (추가 시간 없음),256 MB,수학
254,5258,3489,2,9935,문자열 폭발,"출처,다국어,분류",3672,26433,21.764,12,4.59,2 초 (추가 시간 없음),128 MB,"자료 구조,문자열,스택"
253,5258,2288,2,5577,RBY팡!,"출처,다국어,분류",102,480,29.565,14,3.38,1 초,128 MB,"구현,시뮬레이션"
252,5258,850,2,2098,외판원 순회,분류,3585,22494,28.595,15,3.50,1 초,128 MB,"다이나믹 프로그래밍,비트마스킹,비트필드를 이용한 다이나믹 프로그래밍,외판원 순회 문제"
251,5258,167,2,1167,트리의 지름,분류,4189,15823,35.636,13,2.81,2 초,256 MB,"그래프 이론,그래프 탐색,트리,깊이 우선 탐색"
250,5258,59,2,1059,좋은 구간,분류,1080,5229,25.924,6,3.86,2 초,128 MB,"수학,브루트포스 알고리즘"
167,5258,3992,1,10950,A+B - 3,분류,47583,94066,58.953,3,1.70,1 초,256 MB,"수학,구현,사칙연산"
172,5258,4008,1,10973,이전 순열,분류,3744,7140,65.489,8,1.53,1 초,256 MB,수학
171,5258,4007,1,10972,다음 순열,분류,4909,16119,43.331,8,2.31,1 초,256 MB,"수학,조합론"
170,5258,4006,1,10971,외판원 순회 2,분류,5457,25439,34.747,9,2.88,2 초,256 MB,"백트래킹,외판원 순회 문제"


In [33]:
predictions

,number,name,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit,algos,problem_id,Predictions
3289,9663,N-Queen,분류,12973,36655,53.286,11,1.88,10 초,128 MB,"브루트포스 알고리즘,백트래킹",3437,0.890499
1981,4963,섬의 개수,"출처,다국어,분류",10690,29537,49.204,9,2.03,1 초,128 MB,"그래프 이론,그래프 탐색,너비 우선 탐색,깊이 우선 탐색",2109,0.871751
2707,7562,나이트의 이동,"출처,다국어,분류",8335,23165,46.923,9,2.13,1 초,256 MB,"그래프 이론,그래프 탐색,너비 우선 탐색",2841,0.808738
1111,2583,영역 구하기,"출처,분류",9034,20439,56.615,10,1.77,1 초,128 MB,"그래프 이론,그래프 탐색,너비 우선 탐색,깊이 우선 탐색",1204,0.765276
5816,15552,빠른 A+B,분류,37056,97596,46.940,4,2.13,1 초 (하단 참고),512 MB,"수학,구현,사칙연산",6035,0.738347
143,1158,요세푸스 문제,분류,14146,39824,49.209,6,2.03,2 초,256 MB,"자료 구조,큐",158,0.729984
98,1110,더하기 사이클,분류,44365,111142,48.021,5,2.08,2 초,128 MB,"수학,구현,문자열",110,0.716441
2930,8393,합,"출처,다국어,분류",58154,97631,68.396,1,1.46,1 초,128 MB,"수학,구현",3067,0.714595
2709,7568,덩치,"출처,분류",15518,31086,58.694,6,1.70,1 초,128 MB,"구현,브루트포스 알고리즘",2843,0.688476
3726,10817,세 수,분류,41355,104998,46.811,3,2.14,1 초,256 MB,구현,3890,0.687393


In [34]:
user2idx['kkwinwin95'], user2idx['esun1903'], user2idx['pyob32']

(5258, 10170, 5238)

In [36]:
already_rated_k, predictions_k = recommend_problems(df_svd_preds, 5258, problem_data, score_data, 10)
predictions_k

,number,name,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit,algos,problem_id,Predictions
3289,9663,N-Queen,분류,12973,36655,53.286,11,1.88,10 초,128 MB,"브루트포스 알고리즘,백트래킹",3437,0.890499
1981,4963,섬의 개수,"출처,다국어,분류",10690,29537,49.204,9,2.03,1 초,128 MB,"그래프 이론,그래프 탐색,너비 우선 탐색,깊이 우선 탐색",2109,0.871751
2707,7562,나이트의 이동,"출처,다국어,분류",8335,23165,46.923,9,2.13,1 초,256 MB,"그래프 이론,그래프 탐색,너비 우선 탐색",2841,0.808738
1111,2583,영역 구하기,"출처,분류",9034,20439,56.615,10,1.77,1 초,128 MB,"그래프 이론,그래프 탐색,너비 우선 탐색,깊이 우선 탐색",1204,0.765276
5816,15552,빠른 A+B,분류,37056,97596,46.940,4,2.13,1 초 (하단 참고),512 MB,"수학,구현,사칙연산",6035,0.738347
143,1158,요세푸스 문제,분류,14146,39824,49.209,6,2.03,2 초,256 MB,"자료 구조,큐",158,0.729984
98,1110,더하기 사이클,분류,44365,111142,48.021,5,2.08,2 초,128 MB,"수학,구현,문자열",110,0.716441
2930,8393,합,"출처,다국어,분류",58154,97631,68.396,1,1.46,1 초,128 MB,"수학,구현",3067,0.714595
2709,7568,덩치,"출처,분류",15518,31086,58.694,6,1.70,1 초,128 MB,"구현,브루트포스 알고리즘",2843,0.688476
3726,10817,세 수,분류,41355,104998,46.811,3,2.14,1 초,256 MB,구현,3890,0.687393


In [37]:
# 은선
already_rated_c, predictions_c = recommend_problems(df_svd_preds, 10170, problem_data, score_data, 10)
predictions_c

,number,name,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit,algos,problem_id,Predictions
60,1065,한수,분류,29974,68029,51.984,7,1.92,2 초,128 MB,브루트포스 알고리즘,65,0.777182
1270,2750,수 정렬하기,분류,30976,78735,58.145,5,1.72,1 초,128 MB,"구현,정렬",1335,0.703698
1271,2751,수 정렬하기 2,분류,20294,114927,30.102,6,3.32,2 초,256 MB,정렬,1336,0.692572
1964,4673,셀프 넘버,"출처,다국어,분류",29207,72518,50.260,5,1.99,1 초,256 MB,"수학,구현",2045,0.690241
1397,2941,크로아티아 알파벳,"출처,다국어,분류",20778,54801,45.344,6,2.21,1 초,128 MB,"구현,문자열",1472,0.689267
275,1316,그룹 단어 체커,분류,21685,49239,52.359,6,1.91,2 초,128 MB,"구현,문자열",289,0.661100
4039,11399,ATM,분류,21087,39255,65.971,8,1.52,1 초,256 MB,"그리디 알고리즘,정렬",4168,0.652726
896,2206,벽 부수고 이동하기,분류,7909,52376,22.644,12,4.42,2 초,192 MB,"그래프 이론,그래프 탐색,너비 우선 탐색",925,0.630513
3924,11047,동전 0,"디버그,분류",20005,47672,52.837,9,1.89,1 초,256 MB,그리디 알고리즘,4051,0.628558
708,1931,회의실 배정,분류,15557,73088,28.661,9,3.49,2 초,128 MB,"그리디 알고리즘,정렬",732,0.614110


In [38]:
# 기동형
already_rated_p, predictions_p = recommend_problems(df_svd_preds, 5238, problem_data, score_data, 10)
predictions_p

,number,name,information,correct_user,submission_cnt,correct_rate,level,avg_try,time_limit,memory_limit,algos,problem_id,Predictions
3860,11047,동전 0,"디버그,분류",20005,47672,52.837,9,1.89,1 초,256 MB,그리디 알고리즘,4051,0.868951
5480,14891,톱니바퀴,"디버그,분류",6863,18308,52.469,11,1.91,2 초,512 MB,"구현,시뮬레이션",5711,0.836279
5234,14499,주사위 굴리기,"디버그,분류",7941,26536,41.896,11,2.39,2 초,512 MB,"구현,시뮬레이션",5456,0.830731
2695,7562,나이트의 이동,"출처,다국어,분류",8335,23165,46.923,9,2.13,1 초,256 MB,"그래프 이론,그래프 탐색,너비 우선 탐색",2841,0.823444
4827,13460,구슬 탈출 2,"디버그,분류",7063,46185,25.719,14,3.89,2 초,512 MB,"구현,그래프 이론,그래프 탐색,너비 우선 탐색",5046,0.822167
6257,16234,인구 이동,"디버그,분류",5705,25545,36.225,11,2.76,2 초,512 MB,"구현,그래프 이론,그래프 탐색,너비 우선 탐색,시뮬레이션",6508,0.821823
4300,12100,2048 (Easy),"디버그,분류",6593,43459,24.497,14,4.08,1 초,512 MB,"구현,브루트포스 알고리즘,시뮬레이션",4515,0.801340
146,1158,요세푸스 문제,분류,14146,39824,49.209,6,2.03,2 초,256 MB,"자료 구조,큐",158,0.789961
707,1966,프린터 큐,"출처,다국어,분류",11559,27320,55.296,8,1.81,2 초,128 MB,"구현,자료 구조,시뮬레이션,큐",762,0.778065
1249,2805,나무 자르기,"출처,다국어,분류",11360,63044,25.940,8,3.85,1 초,256 MB,이분 탐색,1373,0.767100
